In [1]:
from math import pow
from polyloss import PolyLoss, PolyFocalLoss
from polynloss import polynloss, polynfocal
import torch
import torch.nn as nn
import torch.nn.functional as F

# Initialize variables

Initializing the variables required for the loss function:<br>
<br>
* n : leading n polynomial coefficients to be used from infinite Taylor series.
* num_classes: number of classes
* ip: the input tensor
* tgt: the actual target values tensor

In [2]:
torch.manual_seed(17)

In [3]:
n = 100
num_classes = 3
ip = torch.rand([10, num_classes])
tgt = torch.Tensor([2, 2, 1, 2, 2, 0, 0, 1, 1, 2]).long()

Apply softmax to input to get probabilities per class

In [4]:
m = nn.Softmax(dim=1)

In [5]:
probs = m(ip)

In [6]:
probs

tensor([[0.2784, 0.3079, 0.4137],
        [0.2907, 0.2645, 0.4449],
        [0.3060, 0.3605, 0.3335],
        [0.2922, 0.2934, 0.4144],
        [0.3177, 0.3136, 0.3687],
        [0.3663, 0.2946, 0.3391],
        [0.3892, 0.3760, 0.2348],
        [0.3079, 0.4051, 0.2870],
        [0.2335, 0.4210, 0.3455],
        [0.2601, 0.2435, 0.4964]])

set class weights as inverse of number of occurances. (alpha parameter in focal loss)

In [7]:
alpha = [0.5, 0.3, 0.2]

# Polyloss

First lets set eps = 0, so that polyloss will be equivalent cross entropy

In [8]:
ploss = PolyLoss(epsilon=0)

In [9]:
pl = ploss(ip, tgt)

Now to cross-validate, we'll use cross_entropy loss from pytorch

In [10]:
ce = F.cross_entropy(ip, tgt)

In [11]:
print("Poly-loss with epsilon 0: ", pl)

Poly-loss with epsilon 0:  tensor(0.9009)


In [12]:
print("Cross entropy loss: ", pl)

Cross entropy loss:  tensor(0.9009)


In [13]:
assert pl.item() - ce.item() < 1e-4 , "cross-entropy loss doesn't match poly-loss with epsilon 0"

Let's check the loss value with polynloss function.

In [14]:
pnloss = polynloss(probs, tgt, [0], reduction='mean')

In [15]:
print("Poly-N-loss with epsilon 0: ", pnloss)

Poly-N-loss with epsilon 0:  tensor(0.9009)


# Poly Focalloss

Let's use epsilon 0 with poly focal-loss so that it'll be equivalent to focal loss

In [16]:
pfocal = PolyFocalLoss(epsilon=0.0, gamma=2.0)

In [17]:
pfl = pfocal(ip, tgt)

In [18]:
polynf = polynfocal(probs, tgt, [0], gamma=2, reduction='mean')

In [19]:
print("Poly-focalloss with epsilon 0: ", pfl)

Poly-focalloss with epsilon 0:  tensor(0.3213)


In [20]:
print("Poly-N-focalloss with epsilon 0: ", polynf)

Poly-N-focalloss with epsilon 0:  tensor(0.3213)


# Poly-loss with epsilon > 0

In [21]:
eps = 0.2

In [22]:
ploss = PolyLoss(epsilon=0.2, reduction='none')

In [23]:
pl = ploss(ip, tgt)

In [24]:
pnloss = polynloss(probs, tgt, [0.2], reduction='none')

In [27]:
print("Instance-wise Poly-loss with epsilon 0.2: ", pl)

Instance-wise Poly-loss with epsilon 0.2:  tensor([1.0000, 0.9210, 1.1482, 0.9982, 1.1239, 1.1310, 1.0658, 1.0226, 0.9809,
        0.8011])


In [28]:
print("Instance-wise Poly-N-loss with epsilon 0.2: ", pnloss)

Instance-wise Poly-N-loss with epsilon 0.2:  tensor([1.0000, 0.9210, 1.1482, 0.9982, 1.1239, 1.1310, 1.0658, 1.0226, 0.9809,
        0.8011])


In [29]:
ploss = PolyLoss(epsilon=0.2, reduction='sum')

In [30]:
pl = ploss(ip, tgt)

In [31]:
pnloss = polynloss(probs, tgt, [0.2], reduction='sum')

In [32]:
print("sum of Poly-loss for all instances: ", pl)

sum of Poly-loss for all instances:  tensor(10.1926)


In [33]:
print("sum of Poly-N-loss for all instances: ", pnloss)

sum of Poly-N-loss for all instances:  tensor(10.1926)


# Poly Focal loss with class weights (alpha)

In [34]:
pfocal = PolyFocalLoss(epsilon=0.2, gamma=2.0, alpha=alpha)

In [35]:
pfl = pfocal(ip, tgt)

In [36]:
print("Poly-focalloss with epsilon 0.2: ", pfl)

Poly-focalloss with epsilon 0.2:  tensor(0.1098)


In [37]:
polynf = polynfocal(probs, tgt, [0.2], gamma=2, reduction='none')

In [42]:
lst = list()
for loss, t in zip(polynf, tgt):
    lst.append(loss.item() * alpha[t])

In [44]:
torch.Tensor(lst).mean()

tensor(0.1098)